In [63]:
import os
import requests #Used to service API connection
from lxml import html #Used to parse XML
from bs4 import BeautifulSoup #Used to read XML table on webpage
import pandas as pd
from pandas.io.json import json_normalize

def makeURL(myUrl, myKey , op, para1, para2):
    # myUrl = "http://192.168.1.120/index.php?"
    url = myUrl + '/'+ op + "?ServiceKey=" + myKey + "&" + 'sigunguCd=' + para1 + '&bjdongCd=' + para2

    url = url.rstrip('&')
    return url

def xmlProcess(url):
    response = requests.get(url)
    # Check if page is up
    if response.status_code == 200:
        # Convert webpage to %Data
        Data = BeautifulSoup(response.text, 'lxml-xml')
        result = []
        rows = 0
        columnName = []
        # search Item all item tag
        iterData = Data.find_all('item')
        for item in iterData:
            item_list = []
            # Fill the value in one row
            for tag in item.find_all():
                try:
                    tagname = tag.name
                    if rows == 0:
                        columnName.append(tagname)
                    item_list.append(item.find(tagname).text)
                except Exception as e:
                    print("This row will be ignored. ", item_list)
            if len(item_list) == 0:
                return pd.DataFrame()
            rows = rows + 1
            result.append(item_list)
#             try:
#                 result.append(item_list)
#             except Exception as e:
#                 return pd.DataFrame()
    finalResult = pd.DataFrame(result)
    finalResult.columns = columnName
#     print(finalResult)
    return finalResult


def jsonProcess(url):

    # 정상 여부 확인 (200 정상)
    response = requests.get(url)
    # JSON 데이터 획득
    json = response.json()
    # PandasDataframe변환
    df = json_normalize(json)
    return df

def csvProcess(url):

    # 정상 여부 확인 (200 정상)
    response = requests.get(url)

    df = pd.read_csv(url, encoding="ms949")
    return df

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)


In [64]:
import requests #Used to service API connection
from lxml import html #Used to parse XML
from bs4 import BeautifulSoup #Used to read XML table on webpage
import pandas as pd
# from common import cFunction as cf
import numpy as np
import wget

# get dataList from filesystem to load and write
#dataList = pd.read_excel("../../data/inbound/dataList.xlsx")

# get dataList from spreadsheet to load and write
dataList = pd.read_csv("https://docs.google.com/spreadsheets/d/1VngqG-m7G8k1587c21MZoheR1Fz3amp1mJtiBvA1Jb0/export?format=csv&gid=0")
print("### The total number of target data is " + str(len(dataList)))

# Filtering -> get dataList only defined url
dataList = dataList[   dataList['사이트'].notnull() ]
print(dataList[["사이트"]])
print("### The total number of filtered data is " + str(len(dataList)))

###################################################
# Filtering -> for your own object
# dataList = dataList[   dataList['번호'] == "339" ]
###################################################

# create folder to save result
outPath = "../../data/outbound/"
folderList = dataList["폴더명"].tolist()
for i in folderList:
    createFolder(outPath+i)

dataList = dataList.fillna("")
dataList = dataList.reset_index(drop=True)

### The total number of target data is 417
                                                   사이트
305  https://www.data.go.kr/dataset/fileDownload.do...
306  http://openapi.kab.co.kr/OpenAPI_ToolInstallPa...
308  http://apis.data.go.kr/1611000/nsdi/BuildingUs...
313         https://data.myhome.go.kr/rentalHouseList?
318  http://openapi.kab.co.kr/OpenAPI_ToolInstallPa...
319  http://openapi.kab.co.kr/OpenAPI_ToolInstallPa...
340      http://apis.data.go.kr/1611000/ArchPmsService
342  http://fsc.go.kr/downManager?bbsid=BBS0069&no=...
345  http://apis.data.go.kr/1611000/ArchPmsService/...
349  http://data.insight.go.kr/openapi/service/Pric...
352  http://openapi.molit.go.kr/OpenAPI_ToolInstall...
353  http://openapi.kab.co.kr/OpenAPI_ToolInstallPa...
354  http://openapi.kab.co.kr/OpenAPI_ToolInstallPa...
355  https://www.data.go.kr/dataset/fileDownload.do...
359  http://openapi.kab.co.kr/OpenAPI_ToolInstallPa...
360  http://api.hf.go.kr:8090/service/rest/HfMbsInt...
361  http://api.hf.go.

In [65]:
# get dataList to load and write
# for dataCount in range(0,len(dataList)):
#     if dataCount == 6:
#         inputUrl = dataList.loc[dataCount, "사이트"]
#         inputKey = dataList.loc[dataCount, "서비스키"]
#         inputParameter = dataList.loc[dataCount, "파라미터"]
#         inputFolder = dataList.loc[dataCount, "폴더명"]
#         inputFile = dataList.loc[dataCount, "서비스명"]
#         inputFile = inputFile.split('&')
#         inputDataType = dataList.loc[dataCount, "데이터타입"]
#         inputRefUrl = dataList.loc[dataCount, "참고문서"]
#         inputRefType = dataList.loc[dataCount, "참고문서타입"]
#     print(inputUrl)

inputUrl = 'http://apis.data.go.kr/1611000/ArchPmsService'
inputKey = 'ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D'
inputFolder = "340_건축인허가 정보"
inputFile = "getApBasisOulnInfo&getApDongOulnInfo&getApFlrOulnInfo&getApHoOulnInfo&getApImprprInfo&getApHdcrMgmRgstInfo&getApDemolExtngMgmRgstInfo&getApTmpBldInfo&getApWclfInfo&getApPklotInfo&getApAtchPklotInfo&getApExposPubuseAreaInfo&getApHoExposPubuseAreaInfo&getApJijiguInfo&getApRoadRgstInfo&getApPlatPlcInfo&getApHsTpInfo"
inputFile = inputFile.split('&')
inputDataType = "xml"
inputRefUrl = 'https://www.data.go.kr/comm/file/download.do?atchFileId=FILE_000000001333976&fileDetailSn=0'
inputRefType = 'hwp'

In [66]:
urlList = []
codeDF = pd.read_csv('../data/inbound/kor_region_code.csv', encoding='ms949')
codeDF['법정동코드'] = codeDF['법정동코드'].astype(str)
len(codeDF)

46092

In [69]:
# newDf = pd.DataFrame()
testList = []
for j in range(len(inputFile)):
    temp = []
    for i in codeDF['법정동코드'].astype(str):
        a = i[:5]
        b = i[5:]
        url = makeURL(inputUrl,inputKey, inputFile[j], a, b)
        temp.append(url)
    testList.append(temp)

In [70]:
outPath = "../data/outbound/"
folderList = inputFolder
createFolder(outPath+folderList)

In [74]:
count = 0

In [ ]:
for i in range(len(testList)):
    tempDF = pd.DataFrame()
    for j in range(len(testList[i])):
        count += 1
        print((count/((len(testList))*(len(testList[i]))))*100)
        try:
            temtem = xmlProcess(testList[i][j])
            tempDF = pd.concat([tempDF, temtem], axis = 0, sort=False)
        except Exception as e:
            print(testList[i][j])        
    fullOutPath = outPath+inputFolder+"/"+ inputFolder + '_' + inputFile[i]+".csv"
    tempDF.to_csv(fullOutPath, encoding='ms949', index=False)

0.041221904018050856
0.04134952601191479
0.04147714800577872
0.04160476999964266
0.0417323919935066
0.04186001398737053
0.04198763598123446
0.0421152579750984
0.04224287996896233
0.04237050196282626
0.042498123956690205
0.042625745950554135
0.042753367944418065
0.04288098993828201
0.04300861193214594
0.04313623392600987
0.04326385591987381
0.04339147791373774
0.04351909990760167
0.04364672190146562
0.04377434389532955
0.04390196588919348
0.044029587883057414
0.04415720987692135
0.04428483187078528
0.04441245386464922
0.044540075858513155
0.044667697852377085
0.04479531984624102
0.04492294184010496
0.04505056383396889
0.045178185827832826
0.04530580782169676
0.04543342981556069
0.04556105180942463
0.04568867380328856
0.0458162957971525
0.045943917791016434
0.046071539784880364
0.0461991617787443
0.04632678377260824
0.04645440576647217
0.046582027760336105
0.04670964975420004
0.04683727174806397
0.04696489374192791
0.047092515735791846
0.047220137729655776
0.04734775972351971
0.047475381

0.0883144197538427
0.08844204174770663
0.08856966374157056
0.0886972857354345
0.08882490772929844
0.08895252972316237
0.08908015171702631
0.08920777371089024
0.08933539570475417
0.08946301769861811
0.08959063969248204
0.08971826168634597
0.08984588368020992
0.08997350567407385
0.09010112766793778
0.09022874966180172
0.09035637165566565
0.09048399364952958
0.09061161564339353
0.09073923763725746
0.09086685963112139
0.09099448162498533
0.09112210361884926
0.09124972561271319
0.09137734760657712
0.09150496960044106
0.091632591594305
0.09176021358816892
0.09188783558203287
0.0920154575758968
0.09214307956976073
0.09227070156362467
0.0923983235574886
0.09252594555135253
0.09265356754521648
0.0927811895390804
0.09290881153294434
0.09303643352680828
0.09316405552067221
0.09329167751453614
0.09341929950840008
0.09354692150226401
0.09367454349612794
0.09380216548999189
0.09392978748385582
0.09405740947771975
0.09418503147158369
0.09431265346544762
0.09444027545931155
0.0945678974531755
0.094695

0.11064826868003123
0.11077589067389519
0.11090351266775911
0.11103113466162304
0.111158756655487
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=11350&bjdongCd=00000
0.11128637864935091
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=11350&bjdongCd=10100
0.11141400064321484
0.1115416226370788
0.11166924463094272
0.11179686662480665
0.1119244886186706
0.11205211061253452
0.11217973260639845
0.11230735460026241
0.11243497659412632
0.11256259858799025
0.11269022058185421
0.11281784257571813
0.11294546456958206
0.11307308656344599
0.11320070855730993
0.11332833055117386
0.11345595254503779
0.11358357453890174
0.11371119653276567
0.1138388185266296
0.11396644052049354
0.11409406251435747
http://apis.data.go.kr/1611000/Ar

0.15199779469194602
0.15212541668580998
0.1522530386796739
0.15238066067353784
0.15250828266740177
0.1526359046612657
0.15276352665512963
0.15289114864899359
0.15301877064285752
0.15314639263672145
0.15327401463058538
0.1534016366244493
0.15352925861831324
0.1536568806121772
0.15378450260604112
0.15391212459990505
0.15403974659376898
0.1541673685876329
0.15429499058149684
0.1544226125753608
0.15455023456922473
0.15467785656308866
0.1548054785569526
0.15493310055081652
0.15506072254468045
0.1551883445385444
0.15531596653240834
0.15544358852627227
0.1555712105201362
0.15569883251400013
0.15582645450786406
0.15595407650172802
0.15608169849559195
0.15620932048945588
0.1563369424833198
0.15646456447718374
0.15659218647104767
0.15671980846491163
0.15684743045877556
0.15697505245263949
0.15710267444650342
0.15723029644036735
0.15735791843423128
0.15748554042809523
0.15761316242195916
0.1577407844158231
0.15786840640968702
0.15799602840355095
0.15812365039741488
0.15825127239127884
0.158378894

0.2050885441393428
0.20521616613320676
0.20534378812707066
0.20547141012093462
0.20559903211479855
0.20572665410866245
0.2058542761025264
0.20598189809639036
0.20610952009025427
0.20623714208411822
0.20636476407798215
0.20649238607184606
0.20662000806571001
0.20674763005957397
0.20687525205343787
0.20700287404730183
0.20713049604116576
0.20725811803502966
0.20738574002889362
0.20751336202275758
0.20764098401662148
0.20776860601048544
0.20789622800434937
0.20802384999821327
0.20815147199207723
0.2082790939859412
0.2084067159798051
0.20853433797366905
0.20866195996753298
0.20878958196139688
0.20891720395526084
0.2090448259491248
0.2091724479429887
0.20930006993685266
0.2094276919307166
0.2095553139245805
0.20968293591844445
0.2098105579123084
0.2099381799061723
0.21006580190003626
0.21019342389390017
0.2103210458877641
0.21044866788162805
0.21057628987549196
0.21070391186935591
0.21083153386321987
0.21095915585708377
0.2110867778509477
0.21121439984481166
0.21134202183867556
0.2114696438

...

1.2674140210627338
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41800&bjdongCd=35032
1.2675416430565978
1.2676692650504617
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41800&bjdongCd=35034
1.2677968870443257
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41800&bjdongCd=35035
1.2679245090381897
1.2680521310320536
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41800&bjdongCd=35037
1.2681797530259176
http://apis.data.go.kr/1611000/ArchPmsServi

1.27507134069457
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41810&bjdongCd=25025
1.2751989626884337
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41810&bjdongCd=25026
1.2753265846822979
1.2754542066761618
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41810&bjdongCd=25321
1.2755818286700256
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41810&bjdongCd=25322
1.2757094506638895
1.2758370726577535
http://apis.data.go.kr/1611000/ArchPmsService

1.2819629283632223
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41810&bjdongCd=35024
1.2820905503570863
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41810&bjdongCd=35025
1.2822181723509503
1.2823457943448142
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41810&bjdongCd=35027
1.2824734163386782
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41810&bjdongCd=35028
1.2826010383325421
1.282728660326406
http://apis.data.go.kr/1611000/ArchPmsServic

1.288854516031875
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41810&bjdongCd=42022
1.2889821380257387
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41810&bjdongCd=42023
1.2891097600196026
1.2892373820134666
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41820&bjdongCd=03000
1.2893650040073306
1.2894926260011945
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41820&bjdongCd=03002
1.2896202479950585
1.2897478699889224
http://apis.data.go.kr/16

1.304424399283275
1.3045520212771389
1.3046796432710026
1.3048072652648668
1.3049348872587307
1.3050625092525945
1.3051901312464584
1.3053177532403224
1.3054453752341864
1.3055729972280503
1.3057006192219143
1.305828241215778
1.3059558632096422
1.3060834852035061
1.3062111071973699
1.3063387291912338
1.3064663511850978
1.3065939731789618
1.3067215951728257
1.3068492171666897
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41830&bjdongCd=32021
1.3069768391605536
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=ufe1cXhfaa3SBtlsFh402kse1ctLxx0%2BxZU0NDXcW6KST%2BIRc2HMgfJMC2%2BJ9vwZqCjCZvIyvusJfRZdESfOIA%3D%3D&sigunguCd=41830&bjdongCd=32022
1.3071044611544176
1.3072320831482815
1.3073597051421453
1.3074873271360092
1.3076149491298734
1.3077425711237372
http://apis.data.go.kr/1611000/ArchPmsService/getApBasisOulnInfo?ServiceKey=u